Our company win a tender in a hospital for a medicine with the **quota = 1000** in the period **from 2020/01/01 to 2021/06/30**. But that particular medicine is not delievered in a single shipment, it can be shiped 30% of the total quota in the first quarter of 2020, another 40% in the final quarter of 2020, and the rest in the 2nd quarter of 2021.

For that specific reason, our clients wants to perform phasing all the tender data by quarter.

For example, the valid period of that contract lasts 561 days.<br>
so in Q1-2020 the phasing value = **1000 (total quota) / 547 (total days in the valid contract) * 90 (total days in Q1'2020) = 164.53**
and that will go on for Q2'2020, Q3'2020, etc. The list goes on as the clients request.

Below is the code to extract the tender data from SQL server and perform phasing by quarter, however I have develop it so you can change it a bit to turn it into phasing by month.

**1. Import neccessary libraries**

In [1]:
import pypyodbc as pyodbc
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import calendar
import tkinter

from sqlalchemy import create_engine # pip install SQLAlchemy
from sqlalchemy.engine import URL
pd.set_option('display.max_columns', None)

**2. Input a list of quarter-year you want to perform phasing**<BR>
ENTER Quarter and year you want to phasing in this format QQ-YYYY (Q1-2023)
E.g. ['Q1-2021','Q2-2021','Q3-2021','Q4-2021']


I have perform a for loop code below, so don't worry about this why you need to create a list variable list this.

In [2]:
times = ['Q1-2021','Q2-2021','Q3-2021','Q4-2021','Q1-2022','Q2-2022','Q3-2022','Q4-2022','Q1-2023','Q2-2023','Q3-2023']

**3. Create connection to extract data from SQL**

In [4]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server={SERVER NAME};"
            "Database={DATABASE NAME};"
            "UID={USER ID};"
            'PWD={PASSWORD};')
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

In [5]:
data = pd.read_sql("SELECT * FROM phasing", cnxn)

C:\AppData\Local\Temp\ipykernel_18252\4151385011.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT * FROM phasing", cnxn)


**4. Data processing**

As you can see the below df, all the columns types are object. We need to convert it into suitable type

In [6]:
data.head(3)

,id,quota(smallestunit),price,resultdate,validto
0,01012020_10382,1000,1680.0,Dec 18 2019,Jun 30 2021
1,01012020_10383,20000,20475.0,Dec 18 2019,Jun 30 2021
2,01012020_10384,1500,1529.0,Dec 18 2019,Jun 30 2021


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016735 entries, 0 to 1016734
Data columns (total 5 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   id                   1016735 non-null  object
 1   quota(smallestunit)  1016115 non-null  object
 2   price                1016701 non-null  object
 3   resultdate           1016735 non-null  object
 4   validto              1016735 non-null  object
dtypes: object(5)
memory usage: 38.8+ MB


In [8]:
#convert quota and price to float type and any error is replace with null values and remove those.
data['quota(smallestunit)'] = pd.to_numeric(data['quota(smallestunit)'], errors='coerce')
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data.dropna(subset=['quota(smallestunit)'])
data.dropna(subset=['price'])

#convert the Result date and the Expired date to datetime column.
data['resultdate_new'] = pd.to_datetime(data['resultdate'])
data['validto_new'] = pd.to_datetime(data['validto'])

Calculate the average quota per day

avg_quota = total_quota / (Expired_date - Result_date + 1)

In [9]:
data['avg_quota'] = data['quota(smallestunit)']/(data['validto_new']-data['resultdate_new']+timedelta(days=1)).dt.days

In [10]:
for time in times:
    year = time[-4:]
    end_date_feb = '29' if int(year)%4 == 0 else '28'
    #generate each quarter to specific start_dates and end_dates of each month in that quarter.
    #for example: Q1-2021. Start_date = ['2023-01-01','2023-02-01','2023-03-01']
    #                      End_date = ['2023-01-31','2023-02-28','2023-03-30']
    if time[:2] == 'Q1':
        startdates = [year + '-01-01',year + '-02-01',year + '-03-01']
        enddates = [year + '-01-31', year + '-02-' + end_date_feb, year + '-03-31']
    elif time [:2] == 'Q2':
        startdates = [year + '-04-01',year + '-05-01',year + '-06-01']
        enddates = [year + '-04-30',year + '-05-31',year +  '-06-30']
    elif time[:2] == 'Q3':
        startdates = [year + '-07-01',year + '-08-01',year + '-09-01']
        enddates = [year + '-07-31',year + '-08-31',year +  '-09-30']
    elif time[:2] == 'Q4':
        startdates = [year + '-10-01',year + '-11-01',year + '-12-01']
        enddates = [year + '-10-31',year + '-11-30', year + '-12-31']
    else:
        tkinter.messagebox.showerror('','you enter the wrong time period, please enter again')
        exit()

    #create a number of columns which are the start date, end date of the quarter in the loop
    #for example: the current loop is at the 'Q1-2023', it will creat 6 columns: df['first_month_start'] = '2023-01-01', df['first_month_end'] = '2023-01-31', etc.
    starts = ['first_month_start','second_month_start','third_month_start']
    ends = ['first_month_end','second_month_end','third_month_end']

    for start, end, s, e in zip(starts, ends, startdates, enddates):
        data[start] = s
        data[end] = e


    #transform those just created columns into datetime column because we assign them with text value in the first place
    for d in data.columns[-6:]:
        data[d] = pd.to_datetime(data[d])


    #create the total quota value of each month in that quarter. So in the end we can sum up all the three-month value as quarter
    phasings_quota = ['first_month_total_quota','second_month_total_quota','third_month_total_quota']

    for start, end, quota in zip(starts,ends,phasings_quota):
        data[quota] = np.where(
            (data['resultdate_new']<=data[start]) & (data['validto_new']>=data[end]),
                data['avg_quota']*(data[end]-data[start]+timedelta(days=1)).dt.days,
            np.where((data['resultdate_new']<=data[start])
                    & ((data[start]<=data['validto_new']) & (data['validto_new']<=data[end])),
                        data['avg_quota']*(data['validto_new']-data[start]+timedelta(days=1)).dt.days,
            np.where(((data[start]<=data['resultdate_new']) & (data['resultdate_new']<=data[end]))
                    & (data['validto_new']>=data[end]),
                        data['avg_quota']*(data[end]-data['resultdate_new']+timedelta(days=1)).dt.days,
                        0
        ))) 
    #just like the above instruction, the following code will add all the three-month value into a quarter value
    data[f'sum_quota_{time}'] = data[phasings_quota[0]] + data[phasings_quota[1]] + data[phasings_quota[2]]
    data[f'sum_value_{time}'] = data['price']*data[f'sum_quota_{time}']

    #we drop the start date, end date, and the total monthly quota column.
    #Note: If you generate phasing by month, you can adjust the this code by remove the sum value of three-month value.
    for s,e,p in zip(starts,ends,phasings_quota):
        data.drop(columns={s,e,p},inplace=True)

Now we random check the value of our new df. Let's use the 0 index in df to calculate


1. number of days = Expiry_date - Result_date + 1 = 2019/12/18 - 2021/06/30 + 1 = 561 (days)
2. average quota per day = total quota / number of days = 1000 / 561 = 1.7825..
3. total quota in Q1-2021 = average quota per day * total days in Q1-2021 = 1.7825 * 90 = 160.4278..

Feel free to check for the rest of the data

In [11]:
data.head()

,id,quota(smallestunit),price,resultdate,validto,resultdate_new,validto_new,avg_quota,sum_quota_Q1-2021,sum_value_Q1-2021,sum_quota_Q2-2021,sum_value_Q2-2021,sum_quota_Q3-2021,sum_value_Q3-2021,sum_quota_Q4-2021,sum_value_Q4-2021,sum_quota_Q1-2022,sum_value_Q1-2022,sum_quota_Q2-2022,sum_value_Q2-2022,sum_quota_Q3-2022,sum_value_Q3-2022,sum_quota_Q4-2022,sum_value_Q4-2022,sum_quota_Q1-2023,sum_value_Q1-2023,sum_quota_Q2-2023,sum_value_Q2-2023,sum_quota_Q3-2023,sum_value_Q3-2023
0,01012020_10382,1000.0,1680.0,Dec 18 2019,Jun 30 2021,2019-12-18,2021-06-30,1.782531,160.427807,2.695187e+05,162.210339,2.725134e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01012020_10383,20000.0,20475.0,Dec 18 2019,Jun 30 2021,2019-12-18,2021-06-30,35.650624,3208.556150,6.569519e+07,3244.206774,6.642513e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01012020_10384,1500.0,1529.0,Dec 18 2019,Jun 30 2021,2019-12-18,2021-06-30,2.673797,240.641711,3.679412e+05,243.315508,3.720294e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01012020_10385,105000.0,102.0,Dec 18 2019,Jun 30 2021,2019-12-18,2021-06-30,187.165775,16844.919786,1.718182e+06,17032.085561,1.737273e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01012020_10386,70000.0,1310.0,Dec 18 2019,Jun 30 2021,2019-12-18,2021-06-30,124.777184,11229.946524,1.471123e+07,11354.723708,1.487469e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': cnxn_str})
enigne = create_engine(connection_url, module=pyodbc)

data.to_sql('phasing_final', enigne, if_exists='replace', index=False)